## COMP541: DEEP LEARNING
## ASSIGNMENT 2: Convolutional Neural Network
### Fall 2022
### Gamze Keçibaş | 60211
---
**TABLE OF CONTENT**     
* ***Convolutional Neural Networks***  
  * Convolutional Filter Receptive Field    
  * Run the Pytorch ConvNet     
  * Add Pooling Layers     
  * Regularize the Network!  
    * Dropout   
    * Weight regularization 
    * Early stopping   
    * Learning rate scheduling 
  * Experiment with the Architecture 
  * Optimize the Architecture 
  * Test the final Architecture on Variations of Data   
* ***Transfer Learning*** 

***1) Convolutional Neural Networks***

In [1]:
import torch
import numpy
import math
from PIL import Image
import requests

In [6]:
ACTORS_TXT = "new_subset_actors.txt"
ACTRESS_TXT = "new_subset_actresses.txt"
actor_data = []
actress_data = []

with open(ACTORS_TXT, "r") as actors:
    for actor in actors:
        split_actor_info = actor.split()
        actor_data.append(split_actor_info)

with open(ACTRESS_TXT, "r") as actresses:
    for actress in actresses:
        split_actress_info = actress.split()
        actress_data.append(split_actress_info)

In [16]:
actor_images = {}
actress_images = {}
       
for actor in actor_data:
    actor_images[actor[-1]] = Image.open(requests.get(actor[4], stream=True).raw)
    
for actress in actress_data:
    actress_images[actress[-1]] = Image.open(requests.get(actress[4], stream=True).raw)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7ff7486498f0>

In [14]:
actress_images

{'962388de77e47001075e2b9d3bcea5067a0d31ef2e3d51e056033fb5f6a23bc0': 'http://static2.wikia.nocookie.net/__cb20100626051952/glee/images/4/4f/Dianna-Agron-dianna-agron-8731364-1937-2560.jpg',
 '1bbcabc38bdbb1f412f14a712dbdfb33bdcbd22330f4a5daecf4d71cbefc990b': 'http://www.hdwallpapersonly.com/wp-content/uploads/2013/06/Dianna-Agron-2.jpg',
 '377da13ce6b487c7c4cd6cd605ba8039effa4c52d59734122d7ecbd019c999cf': 'http://cdn04.cdn.justjared.com/wp-content/uploads/2011/08/agron-letterman/dianna-agron-letterman-02.jpg',
 '85a2ac66d74e450cc2ffb9bc84b4a38b0c724377d37b6577385d88999aff91eb': 'http://cdn01.cdn.justjared.com/wp-content/uploads/2012/09/agron-cite/dianna-agron-salma-hayek-la-cite-du-cinema-launch-02.jpg',
 '0126056b4df4f69504e26b056947998cac5ee5e6e5626ef3941b1725d8723974': 'http://images2.fanpop.com/images/photos/8300000/Dianna-Agron-dianna-agron-8334684-794-1200.jpg',
 '462a174fd8ff562e1d04960da4194ef379710c275dcdf834fe2a713624334a1f': 'http://images2.fanpop.com/image/photos/8800000/di

### PART - 1

In [ ]:
from CnnModel import SimpleCNN
from train import Trainer
from DataLoader import LoaderClass
import torch
from torch.nn import CrossEntropyLoss
from torch import optim
from torchvision import transforms, utils
import numpy as np
import os
from PIL import Image,ImageFile

LR = 1e-4
Momentum = 0.9 # If you use SGD with momentum
BATCH_SIZE = 16
USE_CUDA = True
POOLING = False
NUM_EPOCHS = 200
PATIENCE = -1
TRAIN_PERCENT = 0.8
VAL_PERCENT = 0.2
NUM_ARTISTS = 11
DATA_PATH = "./art_data/artists"
ImageFile.LOAD_TRUNCATED_IMAGES = True # Do not change this


def load_artist_data():
    data = []
    labels = []
    artists = [x for x in os.listdir(DATA_PATH) if x != '.DS_Store']
    print(artists)
    for folder in os.listdir(DATA_PATH):
        class_index = artists.index(folder)
        for image_name in os.listdir(DATA_PATH + "/" + folder):
            img = Image.open(DATA_PATH + "/" + folder + "/" + image_name)
            artist_label = (np.arange(NUM_ARTISTS) == class_index).astype(np.float32)
            data.append(np.array(img))
            labels.append(artist_label)
    shuffler = np.random.permutation(len(labels))
    data = np.array(data)[shuffler]
    labels = np.array(labels)[shuffler]

    length = len(data)
    val_size = int(length*0.2)
    val_data = data[0:val_size+1]
    train_data = data[val_size+1::]
    val_labels = labels[0:val_size+1]
    train_labels = labels[val_size+1::]
    print(val_labels)
    data_dict = {"train_data":train_data,"val_data":val_data}
    label_dict = {"train_labels":np.array(train_labels),"val_labels":np.array(val_labels)}

    return data_dict,label_dict

if __name__ == "__main__":
    data,labels = load_artist_data()
    model = SimpleCNN(use_cuda=True,pooling=POOLING)
    optimizer = optim.Adam(model.parameters(), lr=LR)
    transforms = {
        'train': transforms.Compose([
            transforms.Resize(50),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(50),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        }
    train_dataset = LoaderClass(data,labels,"train",transforms["train"])
    valid_dataset = LoaderClass(data,labels,"val",transforms["val"])

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=4, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(valid_dataset,
                                             batch_size=BATCH_SIZE,
                                             shuffle=True, num_workers=4, pin_memory=True)

    criterion = CrossEntropyLoss()
    trainer_m = Trainer(model, criterion, train_loader, val_loader, optimizer, num_epoch=NUM_EPOCHS, patience=PATIENCE,batch_size=BATCH_SIZE,lr_scheduler= None)
    best_model = trainer_m.train()

['richard wilson', 'paul cezanne', 'paul sandby', 'claude monet', 'peter paul rubens', 'canaletto', 'j. m. w. turner', 'paul gauguin', 'george romney', 'john robert cozens', 'rembrandt']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Epoch 0,phase: Training: 1it [00:00, 36472.21it/s, train_loss : ?, train_acc: ?]/content/drive/MyDrive/COLABS/02-convolutional_neural_network/comp541-hw2-starter-code/train.py:65: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmaxed_scores = self.softmax(logits)
Epoch 1,phase: Training: 2it [00:07,  3.91s/it, val acc:  0.323,val loss:  2.367]  

New best loss, loss is:  2.3667938709259033 acc is:  0.34065934065934067


Epoch 2,phase: Training: 3it [00:08,  2.64s/it, val acc:  0.323,val loss:  2.246]  

New best loss, loss is:  2.246028741200765 acc is:  0.34065934065934067


...

Not a better score


Epoch 199,phase: Training: 200it [03:19,  1.03it/s, val acc:  0.448,val loss:  2.062]  

Not a better score


Epoch 199,phase: Validation: 200it [03:20,  1.00s/it, val acc:  0.448,val loss:  2.058]

Not a better score
Training ended returning the best model
Best val acc: 0.5164835164835165, Best val loss: 2.034157911936442, Best train acc: 0.65, Best train loss: 1.894924241563548 


***1.1) Convolutional Filter Receptive Field***
* **What are the dimensions of the receptive field for a node in Z2? Note that you can ignore padding, and just consider patches in the middle of the image and Z1.** 



* **Why is it effective to build convolutional networks deeper, i.e. with more layers?**

***1.2) Run the Pytorch ConvNet***      
**• How many layers are there? Are they all convolutional? If not, what structure do they have?** 
The best model from run.py has six layers. First one is 2 dimensional convolutional layer, second layer is 2 dimensional max pooling layer, third one is 2 dimensional convolutional, forth one is 2 dimensional max pooling and final two layers are fully connected linear layers.    
  
**• Which activation function is used on the hidden nodes?**  
Rectified Linear Unit (ReLU) is used to on the hidden nodes.  
  
**• What loss function is being used to train the network?**  
Cross Entropy Loss (softmax) is preferred to train the network.  
  
**• How is the loss being minimized?**  
The loss being minimized with zero gradient optimizer.  

In [ ]:
print(best_model)

SimpleCNN(
  (conv_layer1): Conv2d(3, 16, kernel_size=(5, 5), stride=(2, 2))
  (pool_layer1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer2): Conv2d(16, 16, kernel_size=(5, 5), stride=(2, 2))
  (pool_layer2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fully_connected_layer): Linear(in_features=1600, out_features=64, bias=True)
  (final_layer): Linear(in_features=64, out_features=11, bias=True)
)


- **What is the training accuracy for your network after training? What is the validation accuracy?**   
    
End of the default 200 epochs:         
Training ended returning the best model:  
-Best val acc: 0.4725274725274725,     
-Best val loss: 2.042543033758799
    
-Best train acc: 0.6444444444444445,   
-Best train loss: 1.913529396057129  


- **What do these two numbers tell you about what your network is doing?**  
Training accuracy shows that learning success of the best model when the model have whole dataset. Validation accuracy is a result of kind an exam after studying. Validation accuracy provides a learning process for new datasets.  


***1.3 Add Pooling Layers***     
*After you applied max pooling,* 
- What happened to your results?
- How did the training accuracy vs. validation accuracy change? What does that tell you about the effect of max pooling on your network?


In [ ]:
from CnnModel import SimpleCNN
from train import Trainer
from DataLoader import LoaderClass
import torch
from torch.nn import CrossEntropyLoss
from torch import optim
from torchvision import transforms, utils
import numpy as np
import os
from PIL import Image,ImageFile

LR = 1e-4
Momentum = 0.9 # If you use SGD with momentum
BATCH_SIZE = 16
USE_CUDA = True
POOLING = True  ### ADD POOLING LAYER
NUM_EPOCHS = 200
PATIENCE = -1
TRAIN_PERCENT = 0.8
VAL_PERCENT = 0.2
NUM_ARTISTS = 11
DATA_PATH = "./art_data/artists"
ImageFile.LOAD_TRUNCATED_IMAGES = True # Do not change this


def load_artist_data():
    data = []
    labels = []
    artists = [x for x in os.listdir(DATA_PATH) if x != '.DS_Store']
    print(artists)
    for folder in os.listdir(DATA_PATH):
        class_index = artists.index(folder)
        for image_name in os.listdir(DATA_PATH + "/" + folder):
            img = Image.open(DATA_PATH + "/" + folder + "/" + image_name)
            artist_label = (np.arange(NUM_ARTISTS) == class_index).astype(np.float32)
            data.append(np.array(img))
            labels.append(artist_label)
    shuffler = np.random.permutation(len(labels))
    data = np.array(data)[shuffler]
    labels = np.array(labels)[shuffler]

    length = len(data)
    val_size = int(length*0.2)
    val_data = data[0:val_size+1]
    train_data = data[val_size+1::]
    val_labels = labels[0:val_size+1]
    train_labels = labels[val_size+1::]
    print(val_labels)
    data_dict = {"train_data":train_data,"val_data":val_data}
    label_dict = {"train_labels":np.array(train_labels),"val_labels":np.array(val_labels)}

    return data_dict,label_dict

if __name__ == "__main__":
    data,labels = load_artist_data()
    model = SimpleCNN(use_cuda=True,pooling=POOLING)
    optimizer = optim.Adam(model.parameters(), lr=LR)
    transforms = {
        'train': transforms.Compose([
            transforms.Resize(50),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(50),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        }
    train_dataset = LoaderClass(data,labels,"train",transforms["train"])
    valid_dataset = LoaderClass(data,labels,"val",transforms["val"])

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=4, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(valid_dataset,
                                             batch_size=BATCH_SIZE,
                                             shuffle=True, num_workers=4, pin_memory=True)

    criterion = CrossEntropyLoss()
    trainer_m = Trainer(model, criterion, train_loader, val_loader, optimizer, num_epoch=NUM_EPOCHS, patience=PATIENCE,batch_size=BATCH_SIZE,lr_scheduler= None)
    best_model_pool = trainer_m.train()

['richard wilson', 'paul cezanne', 'paul sandby', 'claude monet', 'peter paul rubens', 'canaletto', 'j. m. w. turner', 'paul gauguin', 'george romney', 'john robert cozens', 'rembrandt']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Epoch 1,phase: Training: 2it [00:00,  2.04it/s, val acc:  0.094,val loss:  2.396]  

New best loss, loss is:  2.396190603574117 acc is:  0.0989010989010989


Epoch 2,phase: Training: 3it [00:01,  1.46it/s, val acc:  0.094,val loss:  2.393]  

New best loss, loss is:  2.3928514321645102 acc is:  0.0989010989010989


...

Epoch 37,phase: Training: 38it [00:37,  1.08s/it, val acc:  0.469,val loss:  2.038]  

New best loss, loss is:  2.037895898024241 acc is:  0.4945054945054945


Epoch 38,phase: Training: 39it [00:38,  1.05s/it, val acc:  0.469,val loss:  2.051]  

Not a better score


Epoch 38,phase: Validation: 39it [00:39,  1.05s/it, val acc:  0.500,val loss:  2.025]

In [ ]:
print(best_model_pool)

NameError: ignored

*1.4 Regularize the Network!*

Discuss your results. Which regularization technique was most effective?